# MRI and CSF Mask Processing Playground

## Table of Contents

1. [Introduction](#Introduction)
2. [Setup](#Setup)
3. [Loading and Denoising MRI Data](#Loading-and-Denoising-MRI-Data)
4. [Intensity Normalization](#Intensity-Normalization)
5. [Feature Extraction](#Feature-Extraction)
6. [Clustering with K-Means](#Clustering-with-K-Means)
7. [Morphological Operations](#Morphological-Operations)
8. [Visualization](#Visualization)
9. [Saving Outputs](#Saving-Outputs)
10. [Selecting and Refining the Good Cluster](#Selecting-and-Refining-the-Good-Cluster)
11. [Removing Eyes from the CSF Mask](#Removing-Eyes-from-the-CSF-Mask)
12. [Visualization of Detected Eyes](#Visualization-of-Detected-Eyes)
13. [Saving the Good Cluster Mask](#Saving-the-Good-Cluster-Mask)
14. [Calculating the Volume of CSF](#Calculating-the-Volume-of-CSF)
15. [Final Visualization](#Final-Visualization)
16. [Conclusion](#Conclusion)


## Introduction

Welcome to the **MRI and CSF Mask Processing Playground**! This notebook provides an interactive environment to explore and experiment with the processing pipeline designed for MRI data and CSF mask segmentation. You can run individual cells to execute specific parts of the pipeline, visualize intermediate results, and tweak parameters as needed.

## Setup

### 1. Importing Necessary Libraries

First, ensure that all required libraries are installed. If not, you can install them directly from the notebook using `pip`.

In [ ]:
# Install required packages if not already installed
!pip install -r requirements.txt

### 2. Importing Modules

In [ ]:
import sys
import os

# Add the project directory to the Python path
project_dir = os.path.abspath(os.path.join('..'))  # Adjust the path if necessary
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm

# Import utility functions and configurations
from utils import (
    load_image,
    denoise_image,
    normalize_intensity,
    compute_neighborhood_statistics,
    visualize_feature_distributions,
    analyze_feature_distributions,
    perform_kmeans,
    create_refined_masks,
    plot_refined_masks_on_slices,
    visualize_and_save_html,
    extract_and_decimate_meshes,
    identify_good_cluster,
    save_good_cluster_mask,
    calculate_csf_volume,
    detect_and_remove_eyes,
    visualize_good_csf_mask_html,
    remove_small_objects_refined,
    setup_logging,
    print_memory_usage,
    log_memory_usage,
    compute_z_value,

)
from config import (
    ORIGINAL_MRI_PATH,
    DENOISED_TIFF_PATH,
    MASKS_DIR,
    OUTPUT_HTML,
    OUTPUT_SLICES_DIR,
    K,
    BATCH_SIZE,
    STRUCTURING_ELEMENT_RADIUS,
    MIN_SIZE,
    PLOT_DIR,
    BASE_DIR
)

### 3. Setting Up Logging (Optional)

If you wish to enable logging within the notebook, set the logging parameters accordingly. This can help in monitoring the execution and memory usage.

In [ ]:
# Enable logging if desired
enable_logging = True
log_file = 'logs/execution.log'

if enable_logging:
    import logging
    setup_logging(enable_logging, log_file)

<hr>

## Loading and Denoising MRI Data

### 1. Loading the Original MRI Image

Load the original MRI image using the `load_image` function from `utils.py`.

In [ ]:
# Load the original MRI image
original_img = load_image(ORIGINAL_MRI_PATH)

### 2. Denoising the MRI Image

Denoise the loaded MRI image. If a denoised image already exists, it will be loaded; otherwise, the denoising process will be performed.

In [ ]:
# Denoise the MRI image
denoised_img = denoise_image(original_img, DENOISED_TIFF_PATH)

<hr>

## Intensity Normalization

Normalize the intensity of the denoised MRI image to ensure consistent analysis.

In [ ]:
# Normalize the intensity of the denoised image
denoised_img_before_norm = denoised_img.copy()
denoised_img_normalized = normalize_intensity(denoised_img)

# Free memory
del denoised_img

### 1. Visualizing a Normalized Slice

Plot a specific slice (e.g., slice index 90) to visualize the normalization effect.

In [ ]:
# Define the slice index to visualize
slice_index = 90  # Adjust based on your data

# Plot the normalized slice
if slice_index < denoised_img_normalized.shape[0]:
    plt.figure(figsize=(6, 6))
    plt.imshow(denoised_img_normalized[slice_index], cmap='gray')
    plt.title(f'Normalized Denoised MRI Slice {slice_index}')
    plt.axis('off')
    plt.show()
else:
    print(f"Slice index {slice_index} is out of bounds for image with {denoised_img_normalized.shape[0]} slices.")

<hr>

## Feature Extraction

### 1. Computing Neighborhood Statistics

Compute the neighborhood mean and variance for each voxel to enhance feature representation.

In [ ]:
# Compute neighborhood statistics
neighborhood_size = 3  # 3x3x3 neighborhood
neighborhood_mean, neighborhood_variance = compute_neighborhood_statistics(
    denoised_img_normalized,
    neighborhood_size=neighborhood_size
)

### 2. Preparing Features for Clustering

Flatten and combine the intensity, neighborhood mean, and variance into a feature matrix suitable for K-Means clustering.

In [ ]:
# Prepare features for K-Means clustering with spatial connectivity
print("Preparing features for K-Means clustering with spatial connectivity...")
logging.info("Preparing features for K-Means clustering with spatial connectivity.")

intensity_flat = denoised_img_normalized.flatten().reshape(-1, 1)
mean_flat = neighborhood_mean.flatten().reshape(-1, 1)
variance_flat = neighborhood_variance.flatten().reshape(-1, 1)
z_values = compute_z_value(denoised_img_normalized)

z_flat = z_values.flatten().reshape(-1, 1)

del neighborhood_mean, neighborhood_variance  # Free memory

features = np.hstack((intensity_flat, mean_flat, variance_flat, z_flat)).astype(np.float32)
logging.info(f"Features shape after adding spatial connectivity: {features.shape}")
print(f"Features shape after adding spatial connectivity: {features.shape}")
print_memory_usage()
log_memory_usage()

### 3. Visualizing Feature Distributions

Plot the distributions of the extracted features to understand their characteristics.

In [ ]:
# Visualize feature distributions
visualize_feature_distributions(features)

### 4. Analyzing Feature Relationships

Use Seaborn's pairplot to analyze the relationships between different features.

In [ ]:
# Analyze feature distributions with pairplot
analyze_feature_distributions(features)

<hr>

## Clustering with K-Means

### 1. Performing K-Means Clustering

Apply MiniBatch K-Means to segment the MRI data into distinct clusters.

In [ ]:
# Perform K-Means clustering
kmeans = perform_kmeans(
    features,
    denoised_img_shape=denoised_img_normalized.shape,
    k=K,
    batch_size=BATCH_SIZE,
    connectivity_weight=0.2  # Adjustable parameter
)

# Retrieve cluster labels
labels = kmeans.labels_

# Reshape labels back to the original image dimensions
clustered_img = labels.reshape(denoised_img_normalized.shape)
print("Cluster labels reshaped to image dimensions.")
logging.info("Cluster labels reshaped to image dimensions.")

<hr>

## Morphological Operations

### 1. Creating Refined Masks

Apply morphological operations to each cluster mask to remove small artifacts and fill holes.

In [ ]:
# Define the structuring element
from skimage.morphology import ball

selem = ball(STRUCTURING_ELEMENT_RADIUS)

# Create and refine masks for all clusters
refined_masks, csf_cluster = create_refined_masks(
    clustered_img,
    denoised_img_normalized,
    k=K,
    selem=selem,
    min_size=MIN_SIZE,
    masks_dir=MASKS_DIR
)

# Free memory
del clustered_img

### 2. Visualizing Refined Masks on Slices

Overlay the refined masks onto a specific slice for visualization.

In [ ]:
# Define the slice index to visualize
slice_index = 90  # Adjust based on your data

# Plot refined masks overlaid on the specified slice
plot_refined_masks_on_slices(refined_masks, denoised_img_normalized, slice_index, K)

<hr>

## Visualization

### 1. Visualizing and Saving as HTML

Generate an interactive 3D visualization of the Original MRI, Denoised MRI, and refined CSF masks. Save the visualization as an HTML file for easy sharing and exploration.

In [ ]:
# Visualize and save as HTML
visualize_and_save_html(
    original_img=denoised_img_before_norm,  # Using denoised before normalization as per main.py
    denoised_img=denoised_img_normalized,
    refined_masks=refined_masks,
    k=K,
    output_html=OUTPUT_HTML
)

<hr>

## Saving Outputs

### 1. Saving Specific Slices (Optional)

If you wish to save specific slices with overlayed masks, you can use the `plot_refined_masks_on_slices` function. This step was already performed in the **Morphological Operations** section. However, if you have additional slices to save, you can adjust the slice indices accordingly.

In [ ]:
# Example: Save an additional slice
additional_slice_index = 120  # Adjust based on your data
plot_refined_masks_on_slices(refined_masks, denoised_img_normalized, additional_slice_index, K)

<hr>

## Selecting and Refining the Good Cluster

### 1. Identifying the Good Cluster Based on Density Profile

Select and refine the cluster identified as the CSF (Cerebrospinal Fluid) based on density concentration profiles. This involves analyzing the density distribution along the X and Y axes to determine the most representative cluster for CSF.

In [ ]:
# ----- Step 13: Selecting and Refining the Good Cluster Based on Density Profile -----
logging.info("Selecting and refining the good cluster based on density profile.")
print("\n----- Step 13: Selecting and Refining the Good Cluster -----")

# Identify the good cluster using density profile analysis
good_cluster_idx = identify_good_cluster(
    refined_masks=refined_masks,
    denoised_img_before_norm=denoised_img_before_norm,
    denoised_img_normalized=denoised_img_normalized,
    min_peak_height=1000,  # Adjust based on data characteristics
    histogram_output_path=os.path.join(PLOT_DIR, 'density_histogram.png')
)

# Extract the good cluster mask
good_cluster_mask = refined_masks.get(good_cluster_idx)
if good_cluster_mask is None:
    logging.error(f"Good cluster {good_cluster_idx} mask not found.")
    print(f"Error: Good cluster {good_cluster_idx} mask not found.")
    sys.exit(1)

logging.info(f"Selected good cluster mask with index {good_cluster_idx}.")
print(f"Selected good cluster mask with index {good_cluster_idx}.")

## Removing Eyes from the CSF Mask

### 1. Detecting and Removing Eyes

Detect and remove eye regions from the good CSF mask to ensure that only cerebrospinal fluid is analyzed. This process involves identifying potential eye regions based on size and shape criteria and excluding them from the mask.

In [ ]:
# ----- Step 14: Remove Eyes from the Good Cluster Mask -----
logging.info("Removing eyes from the good cluster mask.")
print("\n----- Step 14: Removing Eyes from the Good Cluster Mask -----")

refined_csf_mask, eyes_centroids, eyes_mask = detect_and_remove_eyes(
    csf_mask=good_cluster_mask,
    min_eye_volume=1000,        # Adjust based on data
    max_eye_volume=8000,        # Adjust based on data
    sphericity_threshold=0.6,   # Adjust based on data
    z_range=(0, 180),            # Adjust based on your MRI data's Z-axis
    y_range=(70, 150),           # Adjust based on your MRI data's Y-axis
    x_range=(30, 100),           # Adjust based on your MRI data's X-axis
    save_eyes_mask=True,
    eyes_mask_path=os.path.join(MASKS_DIR, 'detected_eyes_mask.tif')
)

logging.info(f"Detected eyes at centroids: {eyes_centroids}")
print(f"Detected eyes at centroids: {eyes_centroids}")

## Visualization of Detected Eyes

### 1. Overlaying Detected Eyes on the CSF Mask

Visualize the refined CSF mask with eyes removed by overlaying detected eye regions on a 2D projection. This helps in verifying the accuracy of eye removal and the overall quality of the CSF mask.

In [ ]:
# ----- Step 15: Visualization of Detected Eyes on 2D Projection -----
logging.info("Visualizing detected eyes on 2D projection of the CSF mask.")
print("\n----- Step 15: Visualizing Detected Eyes on 2D Projection -----")

# Create a maximum intensity projection along the Z-axis
projection = refined_csf_mask.max(axis=0)
eyes_projection = eyes_mask.max(axis=0)

plt.figure(figsize=(10, 10))
plt.imshow(projection, cmap='gray')
plt.title('Refined CSF Mask with Eyes Removed')
plt.axis('off')

# Overlay detected eyes with semi-transparent color
plt.imshow(np.ma.masked_where(eyes_projection == 0, eyes_projection), cmap='cool', alpha=0.5)

# Draw circles around detected eyes
ax = plt.gca()
for centroid in eyes_centroids:
    _, y, x = centroid  # Exclude Z for 2D overlay
    circle = plt.Circle((x, y), radius=10, edgecolor='cyan', facecolor='none', linewidth=2)
    ax.add_patch(circle)

plt.tight_layout()
overlaid_path = os.path.join(PLOT_DIR, 'refined_csf_with_eyes_removed.png')
plt.savefig(overlaid_path, dpi=300)
plt.close()

logging.info(f"Saved overlaid refined CSF mask with eyes removed at '{overlaid_path}'.")
print(f"Saved overlaid refined CSF mask with eyes removed at '{overlaid_path}'.")

## Saving the Good Cluster Mask

### 1. Exporting the Refined CSF Mask

Save the refined CSF mask (with eyes removed) as a TIFF file for future analysis and reference.

In [ ]:
# ----- Step 16: Save the Good Cluster Mask as TIFF -----
logging.info("Saving the good cluster mask as TIFF.")
print("\n----- Step 16: Saving the Good Cluster Mask -----")

save_good_cluster_mask(
    mask=refined_csf_mask,
    output_path=os.path.join(MASKS_DIR, "good_csf_mask.tif")
)

## Calculating the Volume of CSF

### 1. Computing CSF Volume

Calculate the total volume of cerebrospinal fluid (CSF) within the MRI scan by applying the refined CSF mask to the original MRI image. This measurement is essential for quantitative analysis and clinical assessments.

In [ ]:
# ----- Step 17: Calculate the Volume of the CSF in the MRI -----
logging.info("Calculating the volume of CSF in the MRI.")
print("\n----- Step 17: Calculating CSF Volume -----")

voxel_dimensions = (0.9765635, 0.9765635, 1.0)  # Example values; adjust as needed

csf_volume_mm3 = calculate_csf_volume(
    csf_mask=refined_csf_mask,
    mri_image=denoised_img_before_norm,
    voxel_dimensions=voxel_dimensions
)

logging.info(f"Total CSF Volume: {csf_volume_mm3} mm³")
print(f"Total CSF Volume: {csf_volume_mm3} mm³")

## Final Visualization

### 1. Interactive 3D Visualization and Export

Generate an interactive 3D visualization of the Good CSF Mask using PyVista. This visualization provides an intuitive understanding of the CSF distribution within the brain and can be exported as an HTML file for sharing and further exploration.

In [ ]:
# ----- Step 18: Final Visualization of the Good CSF Mask -----
logging.info("Final visualization of the Good CSF Mask.")
print("\n----- Step 18: Final Visualization of the Good CSF Mask -----")

visualize_good_csf_mask_html(
    denoised_img=denoised_img_before_norm,
    good_csf_mask=refined_csf_mask,
    output_html=os.path.join(BASE_DIR, "good_csf_mask_visualization.html"),
    step_size=2,
    mc_threshold=0.5,
    decimate_reduction=0.5
)

plot_refined_masks_on_slices(
    refined_masks={good_cluster_idx: refined_csf_mask},  # Only the refined Good CSF mask
    denoised_img_normalized=denoised_img_normalized,
    slice_indices=[slice_index, additional_slice_index],  # Use the slice indices as needed
    k=1,  # K=1 since only one mask is being visualized
    output_dir=OUTPUT_SLICES_DIR
)

## Conclusion

The processing pipeline has successfully loaded and denoised the MRI data, normalized intensities, extracted relevant features, performed clustering, refined masks through morphological operations, identified and refined the good CSF cluster, removed eye regions, calculated the total CSF volume, and generated comprehensive visualizations. These steps facilitate both qualitative and quantitative analyses of cerebrospinal fluid within MRI scans.

Feel free to explore and modify parameters as needed to adapt the pipeline to your specific datasets and research requirements.